In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy as np
from PIL import Image
import argparse
import math

Using TensorFlow backend.
/Users/hupidong/anaconda3/envs/py37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hupidong/anaconda3/envs/py37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hupidong/anaconda3/envs/py37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

整个训练过程可以说判别器 D 和生成器 G 对价值函数 V(G,D) 进行了极小极大化博弈：
![gan_tf_keras1.png](./Res/gan_tf_keras1.png)

### 最优生成器

因为最优的判别器D(x)=P_data/(P_data+P_G)，我们将其代入V（G,D）可得：
![gan_tf_keras2.png](./Res/gan_tf_keras2.png)
该积分进行变换得：
![gan_tf_keras3.png](./Res/gan_tf_keras3.png)
假设存在两个分布 P 和 Q，且这两个分布的平均分布 M=(P+Q)/2，那么这两个分布之间的 JS 散度为 P 与 M 之间的 KL 散度加上 Q 与 M 之间的 KL 散度再除以 2；因此可化为：
![gan_tf_keras4.png](./Res/gan_tf_keras4.png)

JS 散度的取值为 0 到 log2。若两个分布完全没有交集，那么 JS 散度取最大值 log2；若两个分布完全一样，那么 JS 散度取最小值 0。当 P_G=P_data 时，JSD(P_data||P_G) 为 0。综上所述，生成分布当且仅当等于真实数据分布式时，我们可以取得最优生成器。

In [18]:
def generator_model():
    #下面搭建生成器的架构，首先导入序贯模型（sequential），即多个网络层的线性堆叠
    model = Sequential()
    #添加一个全连接层，输入为100维向量，输出为1024维
    model.add(Dense(input_dim=100, units=1024))
    #添加一个激活函数tanh
    model.add(Activation('tanh'))
    #添加一个全连接层，输出为128×7×7维度
    model.add(Dense(128*7*7))
    #添加一个批量归一化层，该层在每个batch上将前一层的激活值重新规范化，即使得其输出数据的均值接近0，其标准差接近1
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    #Reshape层用来将输入shape转换为特定的shape，将含有128*7*7个元素的向量转化为7×7×128张量
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
    #2维上采样层，即将数据的行和列分别重复2次
    model.add(UpSampling2D(size=(2, 2)))
    #添加一个2维卷积层，卷积核大小为5×5，激活函数为tanh，共64个卷积核，并采用padding以保持图像尺寸不变
    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    #卷积核设为1即输出图像的维度
    model.add(Conv2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model

### 最优判别器

原论文中价值函数可写为在 x 上的积分，即将数学期望展开为积分形式：
![gan_tf_keras5.png](./Res/gan_tf_keras5.png)
其实求积分的最大值可以转化为求被积函数的最大值。而求被积函数的最大值是为了求得最优判别器 D，因此不涉及判别器的项都可以看作为常数项。
若令判别器 D(x) 等于 y，那么被积函数可以写为：
![gan_tf_keras6.png](./Res/gan_tf_keras6.png)
为了找到最优的极值点，如果 a+b≠0，我们可以用以下一阶导求解：
![gan_tf_keras6.png](./Res/gan_tf_keras7.png)
因此，最优判别器D(x)=P_data/(P_data+P_G)

In [6]:
def discriminator_model():
    #下面搭建判别器架构，同样采用序贯模型
    model = Sequential()
    
    #添加2维卷积层，卷积核大小为5×5，激活函数为tanh，输入shape在‘channels_first’模式下为（samples,channels，rows，cols）
    #在‘channels_last’模式下为（samples,rows,cols,channels），输出为64维
    model.add(
            Conv2D(64, (5, 5),
            padding='same',
            input_shape=(28, 28, 1))
            )
    model.add(Activation('tanh'))
    #为空域信号施加最大值池化，pool_size取（2，2）代表使图片在两个维度上均变为原长的一半
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #Flatten层把多维输入一维化，常用在从卷积层到全连接层的过渡
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    #一个结点进行二值分类，并采用sigmoid函数的输出作为概念
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [7]:
def generator_containing_discriminator(g, d):
    #将前面定义的生成器架构和判别器架构组拼接成一个大的神经网络，用于判别生成的图片
    model = Sequential()
    #先添加生成器架构，再令d不可训练，即固定d
    #因此在给定d的情况下训练生成器，即通过将生成的结果投入到判别器进行辨别而优化生成器
    model.add(g)
    d.trainable = False
    model.add(d)
    return model

In [8]:
def combine_images(generated_images):
    #生成图片拼接
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image

#### 对于每一次迭代：

-  从真实数据分布 P_data 抽取 m 个样本
-  从先验分布 P_prior(z) 抽取 m 个噪声样本
-  将噪声样本投入 G 而生成数据，即x^tilde = G(Z^i)；通过最大化 V 的近似而更新判别器参数θ_d

以上是学习判别器 D 的过程。因为学习 D 的过程是计算 JS 散度的过程，并且我们希望能最大化价值函数，所以该步骤会重复 k 次。

-  从先验分布 P_prior(z) 中抽取另外 m 个噪声样本 {z^1,...,z^m}
-  通过极小化 V^tilde 而更新生成器参数θ_g

In [27]:
def train(BATCH_SIZE):
    
    # 国内好像不能直接导入数据集，我们试了几次都不行，后来将数据集下载到本地'~/.keras/datasets/'，也就是当前目录（我的是用户文件夹下）下的.keras文件夹中。
    #下载的地址为：https://s3.amazonaws.com/img-datasets/mnist.npz
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    #iamge_data_format选择"channels_last"或"channels_first"，该选项指定了Keras将要使用的维度顺序。
    #"channels_first"假定2D数据的维度顺序为(channels, rows, cols)，3D数据的维度顺序为(channels, conv_dim1, conv_dim2, conv_dim3)
    
    #转换字段类型，并将数据导入变量中
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train[:, :, :, None]
    X_test = X_test[:, :, :, None]
    # X_train = X_train.reshape((X_train.shape, 1) + X_train.shape[1:])
    
    #将定义好的模型架构赋值给特定的变量
    d = discriminator_model()
    g = generator_model()
    d_on_g = generator_containing_discriminator(g, d)
    
    #定义生成器模型判别器模型更新所使用的优化算法及超参数
    d_optim = SGD(lr=0.001, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.001, momentum=0.9, nesterov=True)
    
    #编译三个神经网络并设置损失函数和优化算法，其中损失函数都是用的是二元分类交叉熵函数。编译是用来配置模型学习过程的
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    #前一个架构在固定判别器的情况下训练了生成器，所以在训练判别器之前先要设定其为可训练。
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    
    #下面在满足epoch条件下进行训练
    for epoch in range(30):
        print("Epoch is", epoch)
        
        #计算一个epoch所需要的迭代数量，即训练样本数除批量大小数的值取整；其中shape[0]就是读取矩阵第一维度的长度
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        
        #在一个epoch内进行迭代训练
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            
            #随机生成的噪声服从均匀分布，且采样下界为-1、采样上界为1，输出BATCH_SIZE×100个样本；即抽取一个批量的随机样本
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            
            #抽取一个批量的真实图片
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            
            #生成的图片使用生成器对随机噪声进行推断；verbose为日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录
            generated_images = g.predict(noise, verbose=1)
            
            #每经过100次迭代输出一张生成的图片
            if index % 100 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save(
                    "./GAN/"+str(epoch)+"_"+str(index)+".png")
            
            #将真实的图片和生成的图片以多维数组的形式拼接在一起，真实图片在上，生成图片在下
            X = np.concatenate((image_batch, generated_images))
            
            #生成图片真假标签，即一个包含两倍批量大小的列表；前一个批量大小都是1，代表真实图片，后一个批量大小都是0，代表伪造图片
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            
            #判别器的损失；在一个batch的数据上进行一次参数更新
            d_loss = d.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            
            #随机生成的噪声服从均匀分布
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
            
            #固定判别器
            d.trainable = False
            
            #计算生成器损失；在一个batch的数据上进行一次参数更新
            g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
            
            #令判别器可训练
            d.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            
            #每100次迭代保存一次生成器和判别器的权重
            if index % 100 == 9:
                g.save_weights('generator', True)
                d.save_weights('discriminator', True)

In [28]:
## my own test code
(X_train,y_train),(X_test,y_test)=mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train=X_train[:,:,:,None]
g=generator_model()
g.compile(loss='binary_crossentropy', optimizer="SGD")
batch_size=128
noise=np.random.uniform(-1,1,size=(batch_size,100))
generated_images=g.predict(noise)
image_batch=X_train[0:batch_size]
X=np.concatenate((image_batch,generated_images))

In [29]:
def train_k(BATCH_SIZE,k):
    
    # 国内好像不能直接导入数据集，我们试了几次都不行，后来将数据集下载到本地'~/.keras/datasets/'，也就是当前目录（我的是用户文件夹下）下的.keras文件夹中。
    #下载的地址为：https://s3.amazonaws.com/img-datasets/mnist.npz
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    #iamge_data_format选择"channels_last"或"channels_first"，该选项指定了Keras将要使用的维度顺序。
    #"channels_first"假定2D数据的维度顺序为(channels, rows, cols)，3D数据的维度顺序为(channels, conv_dim1, conv_dim2, conv_dim3)
    
    #转换字段类型，并将数据导入变量中
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train[:, :, :, None]
    X_test = X_test[:, :, :, None]
    # X_train = X_train.reshape((X_train.shape, 1) + X_train.shape[1:])
    
    #将定义好的模型架构赋值给特定的变量
    d = discriminator_model()
    g = generator_model()
    d_on_g = generator_containing_discriminator(g, d)
    
    #定义生成器模型判别器模型更新所使用的优化算法及超参数
    d_optim = SGD(lr=0.001, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.001, momentum=0.9, nesterov=True)
    
    #编译三个神经网络并设置损失函数和优化算法，其中损失函数都是用的是二元分类交叉熵函数。编译是用来配置模型学习过程的
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    #前一个架构在固定判别器的情况下训练了生成器，所以在训练判别器之前先要设定其为可训练。
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    
    #下面在满足epoch条件下进行训练
    for epoch in range(30):
        print("Epoch is", epoch)
        
        #计算一个epoch所需要的迭代数量，即训练样本数除批量大小数的值取整；其中shape[0]就是读取矩阵第一维度的长度
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        
        #在一个epoch内进行迭代训练
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            
            #随机生成的噪声服从均匀分布，且采样下界为-1、采样上界为1，输出BATCH_SIZE×100个样本；即抽取一个批量的随机样本
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            
            #抽取一个批量的真实图片
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            
            #生成的图片使用生成器对随机噪声进行推断；verbose为日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录
            generated_images = g.predict(noise, verbose=1)
            
            #每经过100次迭代输出一张生成的图片
            if index % 100 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save(
                    "./GAN/"+str(epoch)+"_"+str(index)+".png")
            
            #将真实的图片和生成的图片以多维数组的形式拼接在一起，真实图片在上，生成图片在下
            X = np.concatenate((image_batch, generated_images))
            
            #生成图片真假标签，即一个包含两倍批量大小的列表；前一个批量大小都是1，代表真实图片，后一个批量大小都是0，代表伪造图片
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            
            #判别器的损失；在一个batch的数据上进行一次参数更新
            d_loss = d.train_on_batch(X, y)
            #print("batch %d d_loss : %f" % (index, d_loss))
            
            if index % k == 0: 
                #随机生成的噪声服从均匀分布
                noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
                #固定判别器
                d.trainable = False
            
                #计算生成器损失；在一个batch的数据上进行一次参数更新
                g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
            
                #令判别器可训练
                d.trainable = True
                print("batch %d d_loss : %f" % (index, d_loss))
                print("batch %d g_loss : %f" % (index, g_loss))
            
            #每100次迭代保存一次生成器和判别器的权重
            if index % 100 == 9:
                g.save_weights('generator', True)
                d.save_weights('discriminator', True)

In [10]:
def generate(BATCH_SIZE, nice= False ):
    #训练完模型后，可以运行该函数生成图片
    g = generator_model()
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    g.load_weights('generator')
    if nice:
        d = discriminator_model()
        d.compile(loss='binary_crossentropy', optimizer="SGD")
        d.load_weights('discriminator')
        noise = np.random.uniform(-1, 1, (BATCH_SIZE*20, 100))
        generated_images = g.predict(noise, verbose=1)
        d_pret = d.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
        nice_images = nice_images[:, :, :, None]
        for i in range(BATCH_SIZE):
            idx = int(pre_with_index[i][1])
            nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
        image = combine_images(nice_images)
    else:
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        generated_images = g.predict(noise, verbose=0)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(
        "./GAN/generated_image.png")

In [ ]:
train(BATCH_SIZE=36)

In [30]:
train_k(BATCH_SIZE=36,k=10)

Epoch is 0
Number of batches 1666
36/36 [==============================] - 0s 12ms/step
batch 0 d_loss : 0.682336
batch 0 g_loss : 0.735015
36/36 [==============================] - 0s 869us/step
batch 10 d_loss : 0.393709
batch 10 g_loss : 0.828054
36/36 [==============================] - 0s 748us/step
batch 20 d_loss : 0.211775
batch 20 g_loss : 1.292953
36/36 [==============================] - 0s 819us/step
batch 30 d_loss : 0.094821
batch 30 g_loss : 2.072156
36/36 [==============================] - 0s 766us/step
batch 40 d_loss : 0.068646
batch 40 g_loss : 2.627500
36/36 [==============================] - 0s 788us/step
batch 50 d_loss : 0.069032
batch 50 g_loss : 2.873433
36/36 [==============================] - 0s 812us/step
batch 60 d_loss : 0.077229
batch 60 g_loss : 2.920209
36/36 [==============================] - 0s 756us/step
batch 70 d_loss : 0.094962
batch 70 g_loss : 2.775211
36/36 [==============================] - 0s 738us/step
batch 80 d_loss : 0.100059
batch 80 g_loss

36/36 [==============================] - 0s 719us/step
batch 140 d_loss : 0.111333
batch 140 g_loss : 2.630879
36/36 [==============================] - 0s 831us/step
batch 150 d_loss : 0.108486
batch 150 g_loss : 3.155914
36/36 [==============================] - 0s 812us/step
batch 160 d_loss : 0.079475
batch 160 g_loss : 3.474900
36/36 [==============================] - 0s 770us/step
batch 170 d_loss : 0.038735
batch 170 g_loss : 3.724096
36/36 [==============================] - 0s 853us/step
batch 180 d_loss : 0.096762
batch 180 g_loss : 4.000908
36/36 [==============================] - 0s 850us/step
batch 190 d_loss : 0.101024
batch 190 g_loss : 3.884231
36/36 [==============================] - 0s 865us/step
batch 200 d_loss : 0.099127
batch 200 g_loss : 4.090193
36/36 [==============================] - 0s 784us/step
batch 210 d_loss : 0.068449
batch 210 g_loss : 4.486899
36/36 [==============================] - 0s 789us/step
batch 220 d_loss : 0.122193
batch 220 g_loss : 5.272979
3

36/36 [==============================] - 0s 748us/step
batch 270 d_loss : 0.210647
batch 270 g_loss : 3.074482
36/36 [==============================] - 0s 760us/step
batch 280 d_loss : 0.246642
batch 280 g_loss : 2.662836
36/36 [==============================] - 0s 794us/step
batch 290 d_loss : 0.264559
batch 290 g_loss : 2.368301
36/36 [==============================] - 0s 746us/step
batch 300 d_loss : 0.295778
batch 300 g_loss : 1.837642
36/36 [==============================] - 0s 774us/step
batch 310 d_loss : 0.227886
batch 310 g_loss : 2.054160
36/36 [==============================] - 0s 817us/step
batch 320 d_loss : 0.244912
batch 320 g_loss : 1.976597
36/36 [==============================] - 0s 855us/step
batch 330 d_loss : 0.194289
batch 330 g_loss : 2.075638
36/36 [==============================] - 0s 781us/step
batch 340 d_loss : 0.206124
batch 340 g_loss : 1.966317
36/36 [==============================] - 0s 786us/step
batch 350 d_loss : 0.154631
batch 350 g_loss : 2.441333
3

36/36 [==============================] - 0s 792us/step
batch 410 d_loss : 0.197778
batch 410 g_loss : 2.221021
36/36 [==============================] - 0s 795us/step
batch 420 d_loss : 0.123592
batch 420 g_loss : 2.858892
36/36 [==============================] - 0s 1ms/step
batch 430 d_loss : 0.156469
batch 430 g_loss : 2.657044
36/36 [==============================] - 0s 796us/step
batch 440 d_loss : 0.114704
batch 440 g_loss : 2.612553
36/36 [==============================] - 0s 859us/step
batch 450 d_loss : 0.115809
batch 450 g_loss : 2.589263
36/36 [==============================] - 0s 775us/step
batch 460 d_loss : 0.101362
batch 460 g_loss : 2.988387
36/36 [==============================] - 0s 805us/step
batch 470 d_loss : 0.103979
batch 470 g_loss : 2.722085
36/36 [==============================] - 0s 791us/step
batch 480 d_loss : 0.105648
batch 480 g_loss : 2.730220
36/36 [==============================] - 0s 766us/step
batch 490 d_loss : 0.142068
batch 490 g_loss : 2.897335
36/

KeyboardInterrupt: 

In [ ]:
generate(132)